In [7]:
from pathlib import Path
import torch
import torch.backends.cudnn as cudnn
import torch.optim
cudnn.benchmark = True

from models.models import Model
from dataloaders.nyu import NYUDataset
from utils.eval import validate

In [2]:
base_nyu = Path("/DATA/vashchilko/nyudepthv2")
traindir_nyu = base_nyu / 'train'
valdir_nyu = base_nyu / 'val'

assert traindir_nyu.exists(), "Set your own path to train"
assert valdir_nyu.exists(), "Set your own path to val"

In [4]:
val_dataset = NYUDataset(valdir_nyu, split='val')

valloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=1, shuffle=False, num_workers=2
    )

In [9]:
model = Model()
model.load_state_dict(torch.load('./outputs/trained2_state.pth'))
model

Model(
  (conv0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (conv1): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU6(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (4): BatchNorm2

In [11]:
device = 'cuda:1'

avg, img = validate(val_loader=valloader, model=model, epoch=12, device=device)

Test: [300/654]	t_GPU=0.006(0.011)
	RMSE=0.96(0.84) MAE=0.89(0.74) Delta1=0.096(0.448) REL=0.539(0.372) Lg10=0.182(inf) 
Test: [600/654]	t_GPU=0.006(0.009)
	RMSE=0.89(0.81) MAE=0.74(0.70) Delta1=0.656(0.484) REL=0.162(0.346) Lg10=0.079(inf) 

*
RMSE=0.827
MAE=0.717
Delta1=0.480
REL=0.341
Lg10=inf
t_GPU=0.009



In [12]:
device = 'cpu'

avg, img = validate(val_loader=valloader, model=model, epoch=12, device=device)

Test: [300/654]	t_GPU=0.059(0.065)
	RMSE=0.96(0.84) MAE=0.89(0.74) Delta1=0.096(0.448) REL=0.539(0.372) Lg10=0.182(inf) 
Test: [600/654]	t_GPU=0.062(0.064)
	RMSE=0.89(0.81) MAE=0.74(0.70) Delta1=0.656(0.484) REL=0.162(0.346) Lg10=0.079(inf) 

*
RMSE=0.827
MAE=0.717
Delta1=0.480
REL=0.341
Lg10=inf
t_GPU=0.064

